In [ ]:
using Pkg; Pkg.activate(".")
using CairoMakie, LinearAlgebra, LaTeXStrings, FFTA

## Trigonometric Interpolation

This notebook goes through a series of numerical experiments testing the accuracy of trigonometric interpolation.

In [ ]:
"interpolation nodes"
xgrid(N) = [ j * π / N  for j = 0:2N-1 ]

"fourier coefficient indices"
kgrid(N) = [ 0:N; -N+1:-1 ]

"""
construct the coefficients of the trigonometric interpolant
"""
function triginterp_fft(f::Function, N)
    X = xgrid(N)
    # nodal values at interpolation nodes
    F = f.(X) 
    return fft(F) / (2*N)
end 


"""
to evaluate a trigonometric polynomial just sum coefficients * basis
we the take the real part because we assume the function we are 
approximating is real.
"""
evaltrig(x, F̂) = sum( real(F̂k * exp(im * x * k))
                      for (F̂k, k) in zip(F̂, kgrid(length(F̂) ÷ 2)) )
;

We return to our periodic witch of Agnesi example.

In [ ]:
f, α = let c = 3
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

fig1 = Figure(size = (400, 250)); ax1 = Axis(fig1[1,1])
lines!(ax1, -π..π, f); fig1

In [ ]:
NN = [4, 8, 12, 16, 20, 24, 28]
# NN = 8:8:128
xe = range(-π, π, length=1000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    fN = evaltrig.(xe, Ref(F̂))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
fig2 = Figure(size = (400, 300))
ax2 = Axis(fig2[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax2, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax2, NN[4:end], 2*exp.(-α * NN[4:end]), linestyle=:dash, color=:black, label=L"e^{-\alpha N}")
axislegend(ax2, position=:rt)
# fig2
fig2

Now that we have a fast algorithm, we can make the problem a bit harder.

In [ ]:
f, α = let c = 20
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

NN = 16:32:512
xe = range(-π, π, length=4000)
f_ex = f.(xe)
err_max = zeros(length(NN))
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    fN = evaltrig.(xe, Ref(F̂))
    err_max[i] = norm(f_ex - fN, Inf)
end


In [ ]:
fig3 = Figure(size = (400, 300))
ax3 = Axis(fig3[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax3, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax3, NN[4:10], 10*exp.(-α * NN[4:10]), linestyle=:dash, color=:black, label=L"\sim e^{-\alpha N}")
axislegend(ax3, position=:rt)
fig3

To estimate the errors we are evaluating the trigonometric interpolant on an increasing number of grid points. That still has quadratic scaling! But this limitation can also be removed.

### Evaluating Trigonometric Polynomials on a Grid

The key observation is that the IFFT can be used to obtain the nodal values. So to get nodal values on a refined grid we can simply "pad" the vector $\hat{F}$ with zeros. This is what `evaltrig_grid` does.

In [ ]:
function evaltrig_grid(F̂, M::Integer; convert = real)
    N = length(F̂) ÷ 2
    @assert 2 * N == length(F̂)
    @assert M >= N 
    F̂_M = zeros(ComplexF64, 2*M)
    F̂_M[1:N] .= F̂[1:N]
    F̂_M[end-N+1:end] .= F̂[end-N+1:end]
    x = xgrid(M) 
    Fx = real.(ifft(F̂_M) * (2*M))
    return Fx, x
end

In [ ]:
N = 3
F̂ = randn(2*N)

# naive plotting just the 6 nodal values?
xp1 = [xgrid(N); [2*π]]
F1 = real.(ifft(F̂)) * (2*N); push!(F1, F1[1])

# We can of course just use eval_trig on a finer grid
xp2 = range(0, 2*π, length=200)
F2 = evaltrig.(xp2, Ref(F̂))

# Nlog(N) scaling cost 
F3, xp3 = evaltrig_grid(F̂, 200)
push!(F3, F3[1]); xp3 = [xp3; [2*π]]

fig4 = Figure(size = (400, 300)); ax4 = Axis(fig4[1,1])
lines!(ax4, xp1, F1, label = "pw")
lines!(ax4, xp2, F2, label = "evaltrig")
lines!(ax4, xp3, F3, label = "evaltrig_grid", linestyle = :dash)
fig4

Equipped with this technique we can now easily scale the error analysis to arbitarily large problems. 

In [ ]:
f, α = let c = 100
    (x -> 1 / (1 + c^2 * sin(x/2)^2)), 2*asinh(1/c)
end

fig5 = Figure(size = (400, 250)); ax5 = Axis(fig5[1,1])
lines!(ax5, -π..π, f); fig5

In [ ]:

# NN = 16:32:512
NN = 128:128:(16*128)   # 16 * 128 = 2048
err_max = zeros(length(NN))
Nerr = 2^15  # ≈ 32000
for (i, N) in enumerate(NN)
    F̂ = triginterp_fft(f, N);
    Fe, xe = evaltrig_grid(F̂, Nerr)
    err_max[i] = norm(real.(Fe) - f.(xe), Inf)
end


In [ ]:
fig6 = Figure(size = (400, 300))
ax6 = Axis(fig6[1, 1], xlabel = L"N", ylabel = "", yscale = log10)
scatterlines!(ax6, NN, err_max, label=L"\Vert f - I_N f \Vert_\infty")
lines!(ax6, NN[4:10], 10*exp.(-α * NN[4:10]), linestyle=:dash, color=:black, label=L"\sim e^{-\alpha N}")
axislegend(ax6, position=:rt)
fig6

---

### Further Experiments

So far we focused on a very limited range of target functions. Here, we can explore a few more function classes.

In [ ]:
"""
A little utility function to help streamline numerical tests 
- take a function `f` 
- construct the trigonometric interpolant
- evaluate the error at `Nerr` points
- return the error in the max norm.
"""
function interpolation_error(f, N::Integer, Nerr = N * 8)
    F̂ = triginterp_fft(f, N);
    Fe, xe = evaltrig_grid(F̂, Nerr)
    return norm(real.(Fe) - f.(xe), Inf)
end

In [ ]:
# Example: Periodic Fermi-Dirac Function

# define a new function
β = 100   # Fermi - temperature 
f1 = x -> 1 / (1 + exp(β*sin(x)))
α1 = π/β    # NB: only asymptotically exact

# compute errors
NN = 16:16:(10*16); Nerr = 2^12
# NN = 64:64:(10*64); Nerr = 2^14
max_err = interpolation_error.(Ref(f1), NN, Nerr)

# ---------------------------------------------
# plot the target function
fig_1 = Figure(size = (600, 250)) 

ax_11 = Axis(fig_1[1,1], xlabel = "x")
lines!(-π..π, f1, label = L"f")
axislegend(ax_11)

ax_12 = Axis(fig_1[1,2], yscale = log10, xlabel = "N")
# ylims!(ax_32, (1e-11, 1e-3))
scatterlines!(ax_12, NN, max_err, label = L"\Vert f - I_N f \Vert_\infty")
lines!(ax_12, NN[6:end], 2*exp.(-α1 * NN[6:end]), linestyle = :dash, color = :black, label = L"e^{-\alpha N}")
axislegend(ax_12, position=:rt)
fig_1

In [ ]:
# Example: A Singular function C^{2,1} - regularity
#     not analytic so we only get algebraic decay.

# define a new function
f2 = x -> abs(sin(x/2))^3
α2 = 3.0

# compute errors
NN = (2).^(3:2:16); Nerr = 8 * NN[end]
max_err = interpolation_error.(Ref(f2), NN, Nerr)


# # plot the target function
# p1 = plot(f, -π/2, π/2, lw=3, label = "f", xlabel = "x")


# ---------------------------------------------
# plot the target function
fig_2 = Figure(size = (600, 250)) 

ax_21 = Axis(fig_2[1,1], xlabel = "x")
lines!(-π..π, f2, label = L"f")
axislegend(ax_21, position=:ct)

ax_22 = Axis(fig_2[1,2], yscale = log10, xscale = log10, xlabel = "N")
ylims!(ax_22, (1e-12, 1e-3))
scatterlines!(ax_22, NN, max_err, label = L"\Vert f - I_N f \Vert_\infty")
lines!(ax_22, NN[2:5], (NN[2:5]).^(-α2), linestyle = :dash, color = :black, label = L"N^{-\alpha}")
axislegend(ax_22, position=:rt)
fig_2


# # plot errors 
# p2 = plot(NN, max_err; lw=3, m=:o, ms=5, label = L"\|f - I_N f\|_\infty",
#     yscale = :log10, xscale = :log10, xlabel = L"N", 
#     size = (400, 300), ylims = (1e-16, 1.0))
# plot!(p2, NN[4:end], 10*(NN[4:end]).^(-α); lw=2, ls=:dash, c = :black, 
#         label = L"N^{-\alpha}")

# # plot both 
# plot(p1, p2, layout = (1,2), size = (600, 300))


---

In [ ]:
# define a new function
f3 = x -> abs(sin(x/2))^(2+1/3)
α3 = 2+1/3

# compute errors
NN = (2).^(3:2:13); Nerr = 8 * NN[end]
max_err = interpolation_error.(Ref(f3), NN, Nerr)

# ---------------------------------------------
# plot the target function
fig_3 = Figure(size = (600, 250)) 

ax_31 = Axis(fig_3[1,1], xlabel = "x")
lines!(-π..π, f3, label = L"f")
axislegend(ax_31, position=:ct)

ax_32 = Axis(fig_3[1,2], yscale = log10, xscale = log10, xlabel = "N")
ylims!(ax_32, (1e-11, 1e-3))
scatterlines!(ax_32, NN, max_err, label = L"\Vert f - I_N f \Vert_\infty")
lines!(ax_32, NN[3:end], (NN[3:end]).^(-α3), linestyle = :dash, color = :black, label = L"N^{-\alpha}")
axislegend(ax_32, position=:rt)
fig_3

---

In [ ]:
# define a new function
function finf(x)
    if 0 < x < π 
        return exp(-1/sin(x)) 
    else
        return zero(typeof(x))
    end
end

# compute errors, twice: with linear and exponential scaling
NN1 = 8:32:512; Nerr = 8 * NN1[end]
max_err1 = interpolation_error.(Ref(finf), NN1, Nerr)
NN2 = (2).^(2:10); Nerr = 8 * NN2[end]
max_err2 = interpolation_error.(Ref(finf), NN2, Nerr)


# ---------------------------------------------
# plot the target function here 
#   + errors in both linear and log scales!!

fig_4 = Figure(size = (600, 200)) 

ax_41 = Axis(fig_4[1,1], xlabel = "x")
lines!(-π..π, finf, label = L"f")
axislegend(ax_41, position=:lt)

ax_42 = Axis(fig_4[1,2], yscale = log10, xscale = identity, xlabel = "N")
ylims!(ax_42, (1e-14, 1e-1))
scatterlines!(ax_42, NN1, max_err1, label = L"\Vert f - I_N f \Vert_\infty")

ax_43 = Axis(fig_4[1,3], yscale = log10, xscale = log10, xlabel = "N")
ylims!(ax_42, (1e-14, 1e-1))
scatterlines!(ax_43, NN2, max_err2, label = L"\Vert f - I_N f \Vert_\infty")

fig_4